In [ ]:
%matplotlib inline
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc
from matplotlib.colors import LinearSegmentedColormap

data_land_mask = Data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/surfdata_0.9x1.25_hist_78pfts_CMIP6_simyr1850_c201126.nc')
data_mask = data_land_mask.load_variable('PFTDATA_MASK')
data_mask = data_mask[29:, :]

#create func to compute PR
def comp_PR(ctl,scen,q):
    #first get quantile from ctl
    qctl=ctl.quantile(q,dim='time')
    
    del ctl
    gc.collect()
    

    #get subsample scenarios
    qscen=xr.where(scen>qctl,1,0)
    pscen=qscen.sum('time')/len(qscen.time)
    
    del qscen
    gc.collect()

    #compute PR
    PR_scen=pscen/(1-q)
    PR_scen = np.where(data_mask==0, np.nan, PR_scen)
    PR_scen = np.where(PR_scen==0, np.nan, PR_scen)
    
    return PR_scen

def get_data_from_mat(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, :]
    return var

data_surface = Data_from_nc(\
                            '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/' + \
                            'surfdata_irrigation_method.nc'\
                           )   #load the data

data_irrigation_method = data_surface.load_variable('irrigation_method')

data_lon = data_surface.load_variable('LONGXY')
data_lon = data_lon[0,:]
data_lat = data_surface.load_variable('LATIXY')
data_lat = data_lat[29:,0]
data_lat = data_lat[:]
data_time = np.array(range(262800))

def get_data_for_this_study(path, metric):
    qctl_IRR_1901_1930_99 = get_data_from_mat(path + "PercentileValue_IRR_1901_1930_" + \
                     metric + "_99.0%.mat", 'qctl_IRR_1901_1930_99')
    qctl_IRR_1901_1930_99= np.array(qctl_IRR_1901_1930_99)
    qctl_IRR_1901_1930_995 = get_data_from_mat(path + "PercentileValue_IRR_1901_1930_" + \
                     metric + "_99.5%.mat", 'qctl_IRR_1901_1930_995')
    qctl_IRR_1901_1930_995= np.array(qctl_IRR_1901_1930_995)
    qctl_IRR_1901_1930_999 = get_data_from_mat(path + "PercentileValue_IRR_1901_1930_" + \
                     metric + "_99.9%.mat", 'qctl_IRR_1901_1930_999')
    qctl_IRR_1901_1930_999= np.array(qctl_IRR_1901_1930_999)
    
    qctl_IRR_1985_2014_99 = get_data_from_mat(path + "PercentileValue_IRR_1985_2014_" + \
                     metric + "_99.0%.mat", 'qctl_IRR_1985_2014_99')
    qctl_IRR_1985_2014_99= np.array(qctl_IRR_1985_2014_99)
    qctl_IRR_1985_2014_995 = get_data_from_mat(path + "PercentileValue_IRR_1985_2014_" + \
                     metric + "_99.5%.mat", 'qctl_IRR_1985_2014_995')
    qctl_IRR_1985_2014_995= np.array(qctl_IRR_1985_2014_995)
    qctl_IRR_1985_2014_999 = get_data_from_mat(path + "PercentileValue_IRR_1985_2014_" + \
                     metric + "_99.9%.mat", 'qctl_IRR_1985_2014_999')
    qctl_IRR_1985_2014_999= np.array(qctl_IRR_1985_2014_999)
    
    
    
    qctl_NOI_1901_1930_99 = get_data_from_mat(path + "PercentileValue_NOI_1901_1930_" + \
                     metric + "_99.0%.mat", 'qctl_NOI_1901_1930_99')
    qctl_NOI_1901_1930_99= np.array(qctl_NOI_1901_1930_99)
    qctl_NOI_1901_1930_995 = get_data_from_mat(path + "PercentileValue_NOI_1901_1930_" + \
                     metric + "_99.5%.mat", 'qctl_NOI_1901_1930_995')
    qctl_NOI_1901_1930_995= np.array(qctl_NOI_1901_1930_995)
    qctl_NOI_1901_1930_999 = get_data_from_mat(path + "PercentileValue_NOI_1901_1930_" + \
                     metric + "_99.9%.mat", 'qctl_NOI_1901_1930_999')
    qctl_NOI_1901_1930_999= np.array(qctl_NOI_1901_1930_999)
    
    qctl_NOI_1985_2014_99 = get_data_from_mat(path + "PercentileValue_NOI_1985_2014_" + \
                     metric + "_99.0%.mat", 'qctl_NOI_1985_2014_99')
    qctl_NOI_1985_2014_99= np.array(qctl_NOI_1985_2014_99)
    qctl_NOI_1985_2014_995 = get_data_from_mat(path + "PercentileValue_NOI_1985_2014_" + \
                     metric + "_99.5%.mat", 'qctl_NOI_1985_2014_995')
    qctl_NOI_1985_2014_995= np.array(qctl_NOI_1985_2014_995)
    qctl_NOI_1985_2014_999 = get_data_from_mat(path + "PercentileValue_NOI_1985_2014_" + \
                     metric + "_99.9%.mat", 'qctl_NOI_1985_2014_999')
    qctl_NOI_1985_2014_999= np.array(qctl_NOI_1985_2014_999)
    
    return qctl_NOI_1901_1930_99, qctl_NOI_1901_1930_995, qctl_NOI_1901_1930_999, qctl_IRR_1901_1930_99, qctl_IRR_1901_1930_995, qctl_IRR_1901_1930_999, qctl_NOI_1985_2014_99, qctl_NOI_1985_2014_995, qctl_NOI_1985_2014_999, qctl_IRR_1985_2014_99, qctl_IRR_1985_2014_995, qctl_IRR_1985_2014_999

def plot_PR_absolu(ax, data_xarray, title, metric, perc, levels, ticklabels, unit):
    cmap = 'coolwarm'
    Blues = mpl.cm.get_cmap('hot_r')
    
    colors_Blues = [Blues(0.1), Blues(0.3), Blues(0.4), Blues(0.5), Blues(0.6), Blues(0.7), Blues(0.8), Blues(0.9), Blues(1.0)]
    cmap = LinearSegmentedColormap.from_list('cmap_name', colors_Blues, N=9)
    vmin=1/4
    vmax=4
    alpha=0.05
    levels = levels
    im = data_xarray.plot(ax=ax, cmap=cmap,vmin=vmin,vmax=vmax,levels=levels, extend='both', add_colorbar=False, add_labels=False)
    cb = plt.colorbar(im,fraction= 0.3, pad= 0.04, extend='both', orientation='horizontal')
    cb.ax.tick_params(labelsize=12)
    cb.set_ticks(levels)
    cb.set_label(label=metric + ' ' + unit, fontsize=14)
    cb.set_ticklabels(ticklabels)
    ax.coastlines(color='dimgray', linewidth=0.5)
    ax.add_feature(cfeature.OCEAN, color='lightgrey')
    ax.set_title(title, loc='right', fontsize=14)
    ax.set_title(metric + ' ' + perc, loc='left', fontsize=14)
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.1)
    
def plot_PR_single(ax, data_xarray, data_signal, title, metric, perc, unit):
    cmap = 'coolwarm'
    Blues = mpl.cm.get_cmap('coolwarm')
    
    colors_Blues = [Blues(0.0), Blues(0.1), Blues(0.2), Blues(0.3), 'lightgray', Blues(0.7), Blues(0.8), Blues(0.9), Blues(1.0)]
    cmap = LinearSegmentedColormap.from_list('cmap_name', colors_Blues, N=9)
    vmin=1/4
    vmax=4
    alpha=0.05
    levels = [-2, -1, -0.5, -0.2, 0.2, 0.5, 1, 2]
    im = data_xarray.plot(ax=ax, cmap=cmap,vmin=vmin,vmax=vmax,levels=levels, extend='both', add_colorbar=False, add_labels=False)
    cb = plt.colorbar(im,fraction= 0.3, pad= 0.04, extend='both', orientation='horizontal')
    cb.ax.tick_params(labelsize=12)
    cb.set_label(label = '$\\Delta$' + metric + ' ' + unit, fontsize=14)
    cb.set_ticks([-2, -1, -0.5, -0.2, 0.2, 0.5, 1, 2])
    cb.set_ticklabels(["-2", "-1", "-0.5", "-0.2", "+0.2", "+0.5", "+1", "+2"])
    ax.coastlines(color='dimgray', linewidth=0.5)
    ax.add_feature(cfeature.OCEAN, color='lightgrey')
    ax.set_title(title, loc='right', fontsize=14)
    ax.set_title(metric + ' ' + perc, loc='left', fontsize=14)
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.1)
    ax.contourf(data_lon, data_lat, data_signal, levels=[0, 3.5], colors= 'lightgray')
    
def plot_PR_single_no_cb(ax, data_xarray, data_signal, title, metric, perc):
    cmap = 'coolwarm'
    Blues = mpl.cm.get_cmap('coolwarm')
    
    colors_Blues = [Blues(0.0), Blues(0.1), Blues(0.2), Blues(0.3), 'lightgray', Blues(0.7), Blues(0.8), Blues(0.9), Blues(1.0)]
    cmap = LinearSegmentedColormap.from_list('cmap_name', colors_Blues, N=9)
    vmin=1/4
    vmax=4
    alpha=0.05
    levels = [-2, -1, -0.5, -0.2, 0.2, 0.5, 1, 2]
    im = data_xarray.plot(ax=ax, cmap=cmap,vmin=vmin,vmax=vmax,levels=levels, extend='both', add_colorbar=False, add_labels=False)
#     cb = plt.colorbar(im,fraction= 0.3, pad= 0.04, extend='both', orientation='horizontal')
#     cb.ax.tick_params(labelsize=12)
#     cb.set_ticks([-2, -1, -0.5, -0.2, 0.2, 0.5, 1, 2])
#     cb.set_ticklabels(["-2", "-1", "-0.5", "-0.2", "+0.2", "+0.5", "+1", "+2"])
    ax.coastlines(color='dimgray', linewidth=0.5)
    ax.add_feature(cfeature.OCEAN, color='lightgrey')
    ax.set_title(title, loc='right', fontsize=14)
    ax.set_title(metric + ' ' + perc, loc='left', fontsize=14)
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.1)
    ax.contourf(data_lon, data_lat, data_signal, levels=[0, 3.5], colors= 'lightgray')

In [ ]:
def all_for_paper():
    
    metric = 'TSA'
    varname = '$\mathregular{T_{2m}}$'
    unit = '$(^\circ C)$'
    levels = [28+273.15, 30+273.15, 32+273.15, 34+273.15, 36+273.15, 38+273.15, 40+273.15, 42+273.15]
    ticklabels = ["28", "30", "32", "34", "36", "38", "40", "42"]
    
    qctl_NOI_1901_1930_99_CESM2, qctl_NOI_1901_1930_995_CESM2, qctl_NOI_1901_1930_999_CESM2, qctl_IRR_1901_1930_99_CESM2, qctl_IRR_1901_1930_995_CESM2, qctl_IRR_1901_1930_999_CESM2, qctl_NOI_1985_2014_99_CESM2, qctl_NOI_1985_2014_995_CESM2, qctl_NOI_1985_2014_999_CESM2, qctl_IRR_1985_2014_99_CESM2, qctl_IRR_1985_2014_995_CESM2, qctl_IRR_1985_2014_999_CESM2=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/', metric)
    qctl_NOI_1901_1930_99_CESM2_gw, qctl_NOI_1901_1930_995_CESM2_gw, qctl_NOI_1901_1930_999_CESM2_gw, qctl_IRR_1901_1930_99_CESM2_gw, qctl_IRR_1901_1930_995_CESM2_gw, qctl_IRR_1901_1930_999_CESM2_gw, qctl_NOI_1985_2014_99_CESM2_gw, qctl_NOI_1985_2014_995_CESM2_gw, qctl_NOI_1985_2014_999_CESM2_gw, qctl_IRR_1985_2014_99_CESM2_gw, qctl_IRR_1985_2014_995_CESM2_gw, qctl_IRR_1985_2014_999_CESM2_gw=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/', metric)
    qctl_NOI_1901_1930_99_NorESM, qctl_NOI_1901_1930_995_NorESM, qctl_NOI_1901_1930_999_NorESM, qctl_IRR_1901_1930_99_NorESM, qctl_IRR_1901_1930_995_NorESM, qctl_IRR_1901_1930_999_NorESM, qctl_NOI_1985_2014_99_NorESM, qctl_NOI_1985_2014_995_NorESM, qctl_NOI_1985_2014_999_NorESM, qctl_IRR_1985_2014_99_NorESM, qctl_IRR_1985_2014_995_NorESM, qctl_IRR_1985_2014_999_NorESM=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/', metric)
    qctl_NOI_1901_1930_99_E3SM, qctl_NOI_1901_1930_995_E3SM, qctl_NOI_1901_1930_999_E3SM, qctl_IRR_1901_1930_99_E3SM, qctl_IRR_1901_1930_995_E3SM, qctl_IRR_1901_1930_999_E3SM, qctl_NOI_1985_2014_99_E3SM, qctl_NOI_1985_2014_995_E3SM, qctl_NOI_1985_2014_999_E3SM, qctl_IRR_1985_2014_99_E3SM, qctl_IRR_1985_2014_995_E3SM, qctl_IRR_1985_2014_999_E3SM=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/', metric)
    qctl_NOI_1901_1930_99_IPSL, qctl_NOI_1901_1930_995_IPSL, qctl_NOI_1901_1930_999_IPSL, qctl_IRR_1901_1930_99_IPSL, qctl_IRR_1901_1930_995_IPSL, qctl_IRR_1901_1930_999_IPSL, qctl_NOI_1985_2014_99_IPSL, qctl_NOI_1985_2014_995_IPSL, qctl_NOI_1985_2014_999_IPSL, qctl_IRR_1985_2014_99_IPSL, qctl_IRR_1985_2014_995_IPSL, qctl_IRR_1985_2014_999_IPSL=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/', metric)
    qctl_NOI_1901_1930_99_CNRM, qctl_NOI_1901_1930_995_CNRM, qctl_NOI_1901_1930_999_CNRM, qctl_IRR_1901_1930_99_CNRM, qctl_IRR_1901_1930_995_CNRM, qctl_IRR_1901_1930_999_CNRM, qctl_NOI_1985_2014_99_CNRM, qctl_NOI_1985_2014_995_CNRM, qctl_NOI_1985_2014_999_CNRM, qctl_IRR_1985_2014_99_CNRM, qctl_IRR_1985_2014_995_CNRM, qctl_IRR_1985_2014_999_CNRM=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/', metric)
    
    qctl_NOI_1901_1930_999 = (qctl_NOI_1901_1930_999_CESM2 + qctl_NOI_1901_1930_999_CESM2_gw + qctl_NOI_1901_1930_999_NorESM + qctl_NOI_1901_1930_999_E3SM + qctl_NOI_1901_1930_999_IPSL + qctl_NOI_1901_1930_999_CNRM)/6
    
    qctl_NOI_1985_2014_999 = (qctl_NOI_1985_2014_999_CESM2 + qctl_NOI_1985_2014_999_CESM2_gw + qctl_NOI_1985_2014_999_NorESM + qctl_NOI_1985_2014_999_E3SM + qctl_NOI_1985_2014_999_IPSL + qctl_NOI_1985_2014_999_CNRM)/6
    
    qctl_IRR_1901_1930_999 = (qctl_IRR_1901_1930_999_CESM2 + qctl_IRR_1901_1930_999_CESM2_gw + qctl_IRR_1901_1930_999_NorESM + qctl_IRR_1901_1930_999_E3SM + qctl_IRR_1901_1930_999_IPSL + qctl_IRR_1901_1930_999_CNRM)/6
    
    qctl_IRR_1985_2014_999 = (qctl_IRR_1985_2014_999_CESM2 + qctl_IRR_1985_2014_999_CESM2_gw + qctl_IRR_1985_2014_999_NorESM + qctl_IRR_1985_2014_999_E3SM + qctl_IRR_1985_2014_999_IPSL + qctl_IRR_1985_2014_999_CNRM)/6
    
    fig = plt.figure(figsize=(18,16),dpi=300)
    fig.subplots_adjust(hspace=0.05, wspace=0.05, left = 0.05, right = 0.95, top = 0.95, bottom = 0.05)
    proj=ccrs.PlateCarree()


    data_xr_3 = xr.DataArray(qctl_NOI_1901_1930_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(431, projection=proj, frameon=False)
    plot_PR_absolu(ax, data_xr_3, '1901irr (1901-1930)', varname, '99.9%', levels, ticklabels, unit)
    ax.text(0.01, 0.92, 'a', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')
    
    
    
    Oth_999_CNRM = qctl_NOI_1985_2014_999_CNRM - qctl_NOI_1901_1930_999_CNRM
    Oth_999_NorESM = qctl_NOI_1985_2014_999_NorESM - qctl_NOI_1901_1930_999_NorESM
    Oth_999_CESM2 = qctl_NOI_1985_2014_999_CESM2 - qctl_NOI_1901_1930_999_CESM2
    Oth_999_CESM2_gw = qctl_NOI_1985_2014_999_CESM2_gw - qctl_NOI_1901_1930_999_CESM2_gw
    Oth_999_IPSL = qctl_NOI_1985_2014_999_IPSL - qctl_NOI_1901_1930_999_IPSL
    Oth_999_E3SM = qctl_NOI_1985_2014_999_E3SM - qctl_NOI_1901_1930_999_E3SM
    
    Oth_999_consis_pos = np.zeros([163, 288])
    Oth_999_consis_neg = np.zeros([163, 288])
    Oth_999_consis_pos[Oth_999_CNRM>=0.2] = Oth_999_consis_pos[Oth_999_CNRM>=0.2]+1
    Oth_999_consis_neg[Oth_999_CNRM<=-0.2] = Oth_999_consis_neg[Oth_999_CNRM<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_NorESM>=0.2] = Oth_999_consis_pos[Oth_999_NorESM>=0.2]+1
    Oth_999_consis_neg[Oth_999_NorESM<=-0.2] = Oth_999_consis_neg[Oth_999_NorESM<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_CESM2>=0.2] = Oth_999_consis_pos[Oth_999_CESM2>=0.2]+1
    Oth_999_consis_neg[Oth_999_CESM2<=-0.2] = Oth_999_consis_neg[Oth_999_CESM2<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_CESM2_gw>=0.2] = Oth_999_consis_pos[Oth_999_CESM2_gw>=0.2]+1
    Oth_999_consis_neg[Oth_999_CESM2_gw<=-0.2] = Oth_999_consis_neg[Oth_999_CESM2_gw<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_IPSL>=0.2] = Oth_999_consis_pos[Oth_999_IPSL>=0.2]+1
    Oth_999_consis_neg[Oth_999_IPSL<=-0.2] = Oth_999_consis_neg[Oth_999_IPSL<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_E3SM>=0.2] = Oth_999_consis_pos[Oth_999_E3SM>=0.2]+1
    Oth_999_consis_neg[Oth_999_E3SM<=-0.2] = Oth_999_consis_neg[Oth_999_E3SM<=-0.2]+1
    
    Oth_999_consis = np.maximum(Oth_999_consis_pos, Oth_999_consis_neg)
    

    data_xr_6 = xr.DataArray(qctl_NOI_1985_2014_999-qctl_NOI_1901_1930_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(4,3,7, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_6, Oth_999_consis, 'ALL-IE', varname, '99.9%')
    ax.text(0.01, 0.92, 'g', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')

    
    
    Irr_999_CNRM = qctl_IRR_1985_2014_999_CNRM - qctl_NOI_1985_2014_999_CNRM
    Irr_999_NorESM = qctl_IRR_1985_2014_999_NorESM - qctl_NOI_1985_2014_999_NorESM
    Irr_999_CESM2 = qctl_IRR_1985_2014_999_CESM2 - qctl_NOI_1985_2014_999_CESM2
    Irr_999_CESM2_gw = qctl_IRR_1985_2014_999_CESM2_gw - qctl_NOI_1985_2014_999_CESM2_gw
    Irr_999_IPSL = qctl_IRR_1985_2014_999_IPSL - qctl_NOI_1985_2014_999_IPSL
    Irr_999_E3SM = qctl_IRR_1985_2014_999_E3SM - qctl_NOI_1985_2014_999_E3SM
    
    Irr_999_consis_pos = np.zeros([163, 288])
    Irr_999_consis_neg = np.zeros([163, 288])
    Irr_999_consis_pos[Irr_999_CNRM>=0.2] = Irr_999_consis_pos[Irr_999_CNRM>=0.2]+1
    Irr_999_consis_neg[Irr_999_CNRM<=-0.2] = Irr_999_consis_neg[Irr_999_CNRM<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_NorESM>=0.2] = Irr_999_consis_pos[Irr_999_NorESM>=0.2]+1
    Irr_999_consis_neg[Irr_999_NorESM<=-0.2] = Irr_999_consis_neg[Irr_999_NorESM<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_CESM2>=0.2] = Irr_999_consis_pos[Irr_999_CESM2>=0.2]+1
    Irr_999_consis_neg[Irr_999_CESM2<=-0.2] = Irr_999_consis_neg[Irr_999_CESM2<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_CESM2_gw>=0.2] = Irr_999_consis_pos[Irr_999_CESM2_gw>=0.2]+1
    Irr_999_consis_neg[Irr_999_CESM2_gw<=-0.2] = Irr_999_consis_neg[Irr_999_CESM2_gw<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_IPSL>=0.2] = Irr_999_consis_pos[Irr_999_IPSL>=0.2]+1
    Irr_999_consis_neg[Irr_999_IPSL<=-0.2] = Irr_999_consis_neg[Irr_999_IPSL<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_E3SM>=0.2] = Irr_999_consis_pos[Irr_999_E3SM>=0.2]+1
    Irr_999_consis_neg[Irr_999_E3SM<=-0.2] = Irr_999_consis_neg[Irr_999_E3SM<=-0.2]+1
    
    Irr_999_consis = np.maximum(Irr_999_consis_pos, Irr_999_consis_neg)
    

    data_xr_9 = xr.DataArray(qctl_IRR_1985_2014_999-qctl_NOI_1985_2014_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(4,3,10, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_9, Irr_999_consis, 'IE', varname, '99.9%', unit)
    ax.text(0.01, 0.92, 'j', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')
    
    
    
    All_999_CNRM = qctl_IRR_1985_2014_999_CNRM - qctl_IRR_1901_1930_999_CNRM
    All_999_NorESM = qctl_IRR_1985_2014_999_NorESM - qctl_IRR_1901_1930_999_NorESM
    All_999_CESM2 = qctl_IRR_1985_2014_999_CESM2 - qctl_IRR_1901_1930_999_CESM2
    All_999_CESM2_gw = qctl_IRR_1985_2014_999_CESM2_gw - qctl_IRR_1901_1930_999_CESM2_gw
    All_999_IPSL = qctl_IRR_1985_2014_999_IPSL - qctl_IRR_1901_1930_999_IPSL
    All_999_E3SM = qctl_IRR_1985_2014_999_E3SM - qctl_IRR_1901_1930_999_E3SM
    
    All_999_consis_pos = np.zeros([163, 288])
    All_999_consis_neg = np.zeros([163, 288])
    All_999_consis_pos[All_999_CNRM>=0.2] = All_999_consis_pos[All_999_CNRM>=0.2]+1
    All_999_consis_neg[All_999_CNRM<=-0.2] = All_999_consis_neg[All_999_CNRM<=-0.2]+1
    
    All_999_consis_pos[All_999_NorESM>=0.2] = All_999_consis_pos[All_999_NorESM>=0.2]+1
    All_999_consis_neg[All_999_NorESM<=-0.2] = All_999_consis_neg[All_999_NorESM<=-0.2]+1
    
    All_999_consis_pos[All_999_CESM2>=0.2] = All_999_consis_pos[All_999_CESM2>=0.2]+1
    All_999_consis_neg[All_999_CESM2<=-0.2] = All_999_consis_neg[All_999_CESM2<=-0.2]+1
    
    All_999_consis_pos[All_999_CESM2_gw>=0.2] = All_999_consis_pos[All_999_CESM2_gw>=0.2]+1
    All_999_consis_neg[All_999_CESM2_gw<=-0.2] = All_999_consis_neg[All_999_CESM2_gw<=-0.2]+1
    
    All_999_consis_pos[All_999_IPSL>=0.2] = All_999_consis_pos[All_999_IPSL>=0.2]+1
    All_999_consis_neg[All_999_IPSL<=-0.2] = All_999_consis_neg[All_999_IPSL<=-0.2]+1
    
    All_999_consis_pos[All_999_E3SM>=0.2] = All_999_consis_pos[All_999_E3SM>=0.2]+1
    All_999_consis_neg[All_999_E3SM<=-0.2] = All_999_consis_neg[All_999_E3SM<=-0.2]+1
    
    All_999_consis = np.maximum(All_999_consis_pos, All_999_consis_neg)
    

    data_xr_12 = xr.DataArray(qctl_IRR_1985_2014_999-qctl_NOI_1901_1930_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(4,3,4, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_12, All_999_consis, 'ALL', varname, '99.9%')
    ax.text(0.01, 0.92, 'd', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')
    
    
    metric = 'HUMIDEX'
    varname = '$\mathregular{HU}$'
    unit = ''
    levels = [28, 31, 34, 37, 40, 43, 46, 49]
    ticklabels = ["28", "31", "34", "37", "40", "43", "46", "49"]
    
    qctl_NOI_1901_1930_99_CESM2, qctl_NOI_1901_1930_995_CESM2, qctl_NOI_1901_1930_999_CESM2, qctl_IRR_1901_1930_99_CESM2, qctl_IRR_1901_1930_995_CESM2, qctl_IRR_1901_1930_999_CESM2, qctl_NOI_1985_2014_99_CESM2, qctl_NOI_1985_2014_995_CESM2, qctl_NOI_1985_2014_999_CESM2, qctl_IRR_1985_2014_99_CESM2, qctl_IRR_1985_2014_995_CESM2, qctl_IRR_1985_2014_999_CESM2=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/', metric)
    qctl_NOI_1901_1930_99_CESM2_gw, qctl_NOI_1901_1930_995_CESM2_gw, qctl_NOI_1901_1930_999_CESM2_gw, qctl_IRR_1901_1930_99_CESM2_gw, qctl_IRR_1901_1930_995_CESM2_gw, qctl_IRR_1901_1930_999_CESM2_gw, qctl_NOI_1985_2014_99_CESM2_gw, qctl_NOI_1985_2014_995_CESM2_gw, qctl_NOI_1985_2014_999_CESM2_gw, qctl_IRR_1985_2014_99_CESM2_gw, qctl_IRR_1985_2014_995_CESM2_gw, qctl_IRR_1985_2014_999_CESM2_gw=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/', metric)
    qctl_NOI_1901_1930_99_NorESM, qctl_NOI_1901_1930_995_NorESM, qctl_NOI_1901_1930_999_NorESM, qctl_IRR_1901_1930_99_NorESM, qctl_IRR_1901_1930_995_NorESM, qctl_IRR_1901_1930_999_NorESM, qctl_NOI_1985_2014_99_NorESM, qctl_NOI_1985_2014_995_NorESM, qctl_NOI_1985_2014_999_NorESM, qctl_IRR_1985_2014_99_NorESM, qctl_IRR_1985_2014_995_NorESM, qctl_IRR_1985_2014_999_NorESM=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/', metric)
    qctl_NOI_1901_1930_99_E3SM, qctl_NOI_1901_1930_995_E3SM, qctl_NOI_1901_1930_999_E3SM, qctl_IRR_1901_1930_99_E3SM, qctl_IRR_1901_1930_995_E3SM, qctl_IRR_1901_1930_999_E3SM, qctl_NOI_1985_2014_99_E3SM, qctl_NOI_1985_2014_995_E3SM, qctl_NOI_1985_2014_999_E3SM, qctl_IRR_1985_2014_99_E3SM, qctl_IRR_1985_2014_995_E3SM, qctl_IRR_1985_2014_999_E3SM=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/', metric)
    qctl_NOI_1901_1930_99_IPSL, qctl_NOI_1901_1930_995_IPSL, qctl_NOI_1901_1930_999_IPSL, qctl_IRR_1901_1930_99_IPSL, qctl_IRR_1901_1930_995_IPSL, qctl_IRR_1901_1930_999_IPSL, qctl_NOI_1985_2014_99_IPSL, qctl_NOI_1985_2014_995_IPSL, qctl_NOI_1985_2014_999_IPSL, qctl_IRR_1985_2014_99_IPSL, qctl_IRR_1985_2014_995_IPSL, qctl_IRR_1985_2014_999_IPSL=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/', metric)
    qctl_NOI_1901_1930_99_CNRM, qctl_NOI_1901_1930_995_CNRM, qctl_NOI_1901_1930_999_CNRM, qctl_IRR_1901_1930_99_CNRM, qctl_IRR_1901_1930_995_CNRM, qctl_IRR_1901_1930_999_CNRM, qctl_NOI_1985_2014_99_CNRM, qctl_NOI_1985_2014_995_CNRM, qctl_NOI_1985_2014_999_CNRM, qctl_IRR_1985_2014_99_CNRM, qctl_IRR_1985_2014_995_CNRM, qctl_IRR_1985_2014_999_CNRM=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/', metric)
    
    qctl_NOI_1901_1930_999 = (qctl_NOI_1901_1930_999_CESM2 + qctl_NOI_1901_1930_999_CESM2_gw + qctl_NOI_1901_1930_999_NorESM + qctl_NOI_1901_1930_999_E3SM + qctl_NOI_1901_1930_999_IPSL + qctl_NOI_1901_1930_999_CNRM)/6
    
    qctl_NOI_1985_2014_999 = (qctl_NOI_1985_2014_999_CESM2 + qctl_NOI_1985_2014_999_CESM2_gw + qctl_NOI_1985_2014_999_NorESM + qctl_NOI_1985_2014_999_E3SM + qctl_NOI_1985_2014_999_IPSL + qctl_NOI_1985_2014_999_CNRM)/6
    
    qctl_IRR_1901_1930_999 = (qctl_IRR_1901_1930_999_CESM2 + qctl_IRR_1901_1930_999_CESM2_gw + qctl_IRR_1901_1930_999_NorESM + qctl_IRR_1901_1930_999_E3SM + qctl_IRR_1901_1930_999_IPSL + qctl_IRR_1901_1930_999_CNRM)/6
    
    qctl_IRR_1985_2014_999 = (qctl_IRR_1985_2014_999_CESM2 + qctl_IRR_1985_2014_999_CESM2_gw + qctl_IRR_1985_2014_999_NorESM + qctl_IRR_1985_2014_999_E3SM + qctl_IRR_1985_2014_999_IPSL + qctl_IRR_1985_2014_999_CNRM)/6
    



    data_xr_3 = xr.DataArray(qctl_NOI_1901_1930_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(432, projection=proj, frameon=False)
    plot_PR_absolu(ax, data_xr_3, '1901irr (1901-1930)', varname, '99.9%', levels, ticklabels, unit)
    ax.text(0.01, 0.92, 'b', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')
    
    
    
    Oth_999_CNRM = qctl_NOI_1985_2014_999_CNRM - qctl_NOI_1901_1930_999_CNRM
    Oth_999_NorESM = qctl_NOI_1985_2014_999_NorESM - qctl_NOI_1901_1930_999_NorESM
    Oth_999_CESM2 = qctl_NOI_1985_2014_999_CESM2 - qctl_NOI_1901_1930_999_CESM2
    Oth_999_CESM2_gw = qctl_NOI_1985_2014_999_CESM2_gw - qctl_NOI_1901_1930_999_CESM2_gw
    Oth_999_IPSL = qctl_NOI_1985_2014_999_IPSL - qctl_NOI_1901_1930_999_IPSL
    Oth_999_E3SM = qctl_NOI_1985_2014_999_E3SM - qctl_NOI_1901_1930_999_E3SM
    
    Oth_999_consis_pos = np.zeros([163, 288])
    Oth_999_consis_neg = np.zeros([163, 288])
    Oth_999_consis_pos[Oth_999_CNRM>=0.2] = Oth_999_consis_pos[Oth_999_CNRM>=0.2]+1
    Oth_999_consis_neg[Oth_999_CNRM<=-0.2] = Oth_999_consis_neg[Oth_999_CNRM<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_NorESM>=0.2] = Oth_999_consis_pos[Oth_999_NorESM>=0.2]+1
    Oth_999_consis_neg[Oth_999_NorESM<=-0.2] = Oth_999_consis_neg[Oth_999_NorESM<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_CESM2>=0.2] = Oth_999_consis_pos[Oth_999_CESM2>=0.2]+1
    Oth_999_consis_neg[Oth_999_CESM2<=-0.2] = Oth_999_consis_neg[Oth_999_CESM2<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_CESM2_gw>=0.2] = Oth_999_consis_pos[Oth_999_CESM2_gw>=0.2]+1
    Oth_999_consis_neg[Oth_999_CESM2_gw<=-0.2] = Oth_999_consis_neg[Oth_999_CESM2_gw<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_IPSL>=0.2] = Oth_999_consis_pos[Oth_999_IPSL>=0.2]+1
    Oth_999_consis_neg[Oth_999_IPSL<=-0.2] = Oth_999_consis_neg[Oth_999_IPSL<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_E3SM>=0.2] = Oth_999_consis_pos[Oth_999_E3SM>=0.2]+1
    Oth_999_consis_neg[Oth_999_E3SM<=-0.2] = Oth_999_consis_neg[Oth_999_E3SM<=-0.2]+1
    
    Oth_999_consis = np.maximum(Oth_999_consis_pos, Oth_999_consis_neg)
    

    data_xr_6 = xr.DataArray(qctl_NOI_1985_2014_999-qctl_NOI_1901_1930_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(4, 3, 8, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_6, Oth_999_consis, 'ALL-IE', varname, '99.9%')
    ax.text(0.01, 0.92, 'h', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')

    
    
    Irr_999_CNRM = qctl_IRR_1985_2014_999_CNRM - qctl_NOI_1985_2014_999_CNRM
    Irr_999_NorESM = qctl_IRR_1985_2014_999_NorESM - qctl_NOI_1985_2014_999_NorESM
    Irr_999_CESM2 = qctl_IRR_1985_2014_999_CESM2 - qctl_NOI_1985_2014_999_CESM2
    Irr_999_CESM2_gw = qctl_IRR_1985_2014_999_CESM2_gw - qctl_NOI_1985_2014_999_CESM2_gw
    Irr_999_IPSL = qctl_IRR_1985_2014_999_IPSL - qctl_NOI_1985_2014_999_IPSL
    Irr_999_E3SM = qctl_IRR_1985_2014_999_E3SM - qctl_NOI_1985_2014_999_E3SM
    
    Irr_999_consis_pos = np.zeros([163, 288])
    Irr_999_consis_neg = np.zeros([163, 288])
    Irr_999_consis_pos[Irr_999_CNRM>=0.2] = Irr_999_consis_pos[Irr_999_CNRM>=0.2]+1
    Irr_999_consis_neg[Irr_999_CNRM<=-0.2] = Irr_999_consis_neg[Irr_999_CNRM<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_NorESM>=0.2] = Irr_999_consis_pos[Irr_999_NorESM>=0.2]+1
    Irr_999_consis_neg[Irr_999_NorESM<=-0.2] = Irr_999_consis_neg[Irr_999_NorESM<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_CESM2>=0.2] = Irr_999_consis_pos[Irr_999_CESM2>=0.2]+1
    Irr_999_consis_neg[Irr_999_CESM2<=-0.2] = Irr_999_consis_neg[Irr_999_CESM2<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_CESM2_gw>=0.2] = Irr_999_consis_pos[Irr_999_CESM2_gw>=0.2]+1
    Irr_999_consis_neg[Irr_999_CESM2_gw<=-0.2] = Irr_999_consis_neg[Irr_999_CESM2_gw<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_IPSL>=0.2] = Irr_999_consis_pos[Irr_999_IPSL>=0.2]+1
    Irr_999_consis_neg[Irr_999_IPSL<=-0.2] = Irr_999_consis_neg[Irr_999_IPSL<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_E3SM>=0.2] = Irr_999_consis_pos[Irr_999_E3SM>=0.2]+1
    Irr_999_consis_neg[Irr_999_E3SM<=-0.2] = Irr_999_consis_neg[Irr_999_E3SM<=-0.2]+1
    
    Irr_999_consis = np.maximum(Irr_999_consis_pos, Irr_999_consis_neg)
    

    data_xr_9 = xr.DataArray(qctl_IRR_1985_2014_999-qctl_NOI_1985_2014_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(4,3,11, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_9, Irr_999_consis, 'IE', varname, '99.9%', unit)
    ax.text(0.01, 0.92, 'k', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')
    
    
    
    All_999_CNRM = qctl_IRR_1985_2014_999_CNRM - qctl_IRR_1901_1930_999_CNRM
    All_999_NorESM = qctl_IRR_1985_2014_999_NorESM - qctl_IRR_1901_1930_999_NorESM
    All_999_CESM2 = qctl_IRR_1985_2014_999_CESM2 - qctl_IRR_1901_1930_999_CESM2
    All_999_CESM2_gw = qctl_IRR_1985_2014_999_CESM2_gw - qctl_IRR_1901_1930_999_CESM2_gw
    All_999_IPSL = qctl_IRR_1985_2014_999_IPSL - qctl_IRR_1901_1930_999_IPSL
    All_999_E3SM = qctl_IRR_1985_2014_999_E3SM - qctl_IRR_1901_1930_999_E3SM
    
    All_999_consis_pos = np.zeros([163, 288])
    All_999_consis_neg = np.zeros([163, 288])
    All_999_consis_pos[All_999_CNRM>=0.2] = All_999_consis_pos[All_999_CNRM>=0.2]+1
    All_999_consis_neg[All_999_CNRM<=-0.2] = All_999_consis_neg[All_999_CNRM<=-0.2]+1
    
    All_999_consis_pos[All_999_NorESM>=0.2] = All_999_consis_pos[All_999_NorESM>=0.2]+1
    All_999_consis_neg[All_999_NorESM<=-0.2] = All_999_consis_neg[All_999_NorESM<=-0.2]+1
    
    All_999_consis_pos[All_999_CESM2>=0.2] = All_999_consis_pos[All_999_CESM2>=0.2]+1
    All_999_consis_neg[All_999_CESM2<=-0.2] = All_999_consis_neg[All_999_CESM2<=-0.2]+1
    
    All_999_consis_pos[All_999_CESM2_gw>=0.2] = All_999_consis_pos[All_999_CESM2_gw>=0.2]+1
    All_999_consis_neg[All_999_CESM2_gw<=-0.2] = All_999_consis_neg[All_999_CESM2_gw<=-0.2]+1
    
    All_999_consis_pos[All_999_IPSL>=0.2] = All_999_consis_pos[All_999_IPSL>=0.2]+1
    All_999_consis_neg[All_999_IPSL<=-0.2] = All_999_consis_neg[All_999_IPSL<=-0.2]+1
    
    All_999_consis_pos[All_999_E3SM>=0.2] = All_999_consis_pos[All_999_E3SM>=0.2]+1
    All_999_consis_neg[All_999_E3SM<=-0.2] = All_999_consis_neg[All_999_E3SM<=-0.2]+1
    
    All_999_consis = np.maximum(All_999_consis_pos, All_999_consis_neg)
    

    data_xr_12 = xr.DataArray(qctl_IRR_1985_2014_999-qctl_NOI_1901_1930_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(4,3,5, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_12, All_999_consis, 'ALL', varname, '99.9%')
    ax.text(0.01, 0.92, 'e', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')
    
    
    
    
    
    metric = 'T_w'
    varname = '$\mathregular{T_{w}}$'
    unit = '$(^\circ C)$'
    levels = [17, 19, 21, 23, 25, 27, 29, 31]
    ticklabels = ["17", "19", "24", "26", "25", "27", "29", "31"]
    
    qctl_NOI_1901_1930_99_CESM2, qctl_NOI_1901_1930_995_CESM2, qctl_NOI_1901_1930_999_CESM2, qctl_IRR_1901_1930_99_CESM2, qctl_IRR_1901_1930_995_CESM2, qctl_IRR_1901_1930_999_CESM2, qctl_NOI_1985_2014_99_CESM2, qctl_NOI_1985_2014_995_CESM2, qctl_NOI_1985_2014_999_CESM2, qctl_IRR_1985_2014_99_CESM2, qctl_IRR_1985_2014_995_CESM2, qctl_IRR_1985_2014_999_CESM2=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/', metric)
    qctl_NOI_1901_1930_99_CESM2_gw, qctl_NOI_1901_1930_995_CESM2_gw, qctl_NOI_1901_1930_999_CESM2_gw, qctl_IRR_1901_1930_99_CESM2_gw, qctl_IRR_1901_1930_995_CESM2_gw, qctl_IRR_1901_1930_999_CESM2_gw, qctl_NOI_1985_2014_99_CESM2_gw, qctl_NOI_1985_2014_995_CESM2_gw, qctl_NOI_1985_2014_999_CESM2_gw, qctl_IRR_1985_2014_99_CESM2_gw, qctl_IRR_1985_2014_995_CESM2_gw, qctl_IRR_1985_2014_999_CESM2_gw=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/', metric)
    qctl_NOI_1901_1930_99_NorESM, qctl_NOI_1901_1930_995_NorESM, qctl_NOI_1901_1930_999_NorESM, qctl_IRR_1901_1930_99_NorESM, qctl_IRR_1901_1930_995_NorESM, qctl_IRR_1901_1930_999_NorESM, qctl_NOI_1985_2014_99_NorESM, qctl_NOI_1985_2014_995_NorESM, qctl_NOI_1985_2014_999_NorESM, qctl_IRR_1985_2014_99_NorESM, qctl_IRR_1985_2014_995_NorESM, qctl_IRR_1985_2014_999_NorESM=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/', metric)
    qctl_NOI_1901_1930_99_E3SM, qctl_NOI_1901_1930_995_E3SM, qctl_NOI_1901_1930_999_E3SM, qctl_IRR_1901_1930_99_E3SM, qctl_IRR_1901_1930_995_E3SM, qctl_IRR_1901_1930_999_E3SM, qctl_NOI_1985_2014_99_E3SM, qctl_NOI_1985_2014_995_E3SM, qctl_NOI_1985_2014_999_E3SM, qctl_IRR_1985_2014_99_E3SM, qctl_IRR_1985_2014_995_E3SM, qctl_IRR_1985_2014_999_E3SM=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/', metric)
    qctl_NOI_1901_1930_99_IPSL, qctl_NOI_1901_1930_995_IPSL, qctl_NOI_1901_1930_999_IPSL, qctl_IRR_1901_1930_99_IPSL, qctl_IRR_1901_1930_995_IPSL, qctl_IRR_1901_1930_999_IPSL, qctl_NOI_1985_2014_99_IPSL, qctl_NOI_1985_2014_995_IPSL, qctl_NOI_1985_2014_999_IPSL, qctl_IRR_1985_2014_99_IPSL, qctl_IRR_1985_2014_995_IPSL, qctl_IRR_1985_2014_999_IPSL=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/', metric)
    qctl_NOI_1901_1930_99_CNRM, qctl_NOI_1901_1930_995_CNRM, qctl_NOI_1901_1930_999_CNRM, qctl_IRR_1901_1930_99_CNRM, qctl_IRR_1901_1930_995_CNRM, qctl_IRR_1901_1930_999_CNRM, qctl_NOI_1985_2014_99_CNRM, qctl_NOI_1985_2014_995_CNRM, qctl_NOI_1985_2014_999_CNRM, qctl_IRR_1985_2014_99_CNRM, qctl_IRR_1985_2014_995_CNRM, qctl_IRR_1985_2014_999_CNRM=get_data_for_this_study('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/', metric)
    
    qctl_NOI_1901_1930_999 = (qctl_NOI_1901_1930_999_CESM2 + qctl_NOI_1901_1930_999_CESM2_gw + qctl_NOI_1901_1930_999_NorESM + qctl_NOI_1901_1930_999_E3SM + qctl_NOI_1901_1930_999_IPSL + qctl_NOI_1901_1930_999_CNRM)/6
    
    qctl_NOI_1985_2014_999 = (qctl_NOI_1985_2014_999_CESM2 + qctl_NOI_1985_2014_999_CESM2_gw + qctl_NOI_1985_2014_999_NorESM + qctl_NOI_1985_2014_999_E3SM + qctl_NOI_1985_2014_999_IPSL + qctl_NOI_1985_2014_999_CNRM)/6
    
    qctl_IRR_1901_1930_999 = (qctl_IRR_1901_1930_999_CESM2 + qctl_IRR_1901_1930_999_CESM2_gw + qctl_IRR_1901_1930_999_NorESM + qctl_IRR_1901_1930_999_E3SM + qctl_IRR_1901_1930_999_IPSL + qctl_IRR_1901_1930_999_CNRM)/6
    
    qctl_IRR_1985_2014_999 = (qctl_IRR_1985_2014_999_CESM2 + qctl_IRR_1985_2014_999_CESM2_gw + qctl_IRR_1985_2014_999_NorESM + qctl_IRR_1985_2014_999_E3SM + qctl_IRR_1985_2014_999_IPSL + qctl_IRR_1985_2014_999_CNRM)/6
    



    data_xr_3 = xr.DataArray(qctl_NOI_1901_1930_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(433, projection=proj, frameon=False)
    plot_PR_absolu(ax, data_xr_3, '1901irr (1901-1930)', varname, '99.9%', levels, ticklabels, unit)
    ax.text(0.01, 0.92, 'c', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')
    
    
    
    Oth_999_CNRM = qctl_NOI_1985_2014_999_CNRM - qctl_NOI_1901_1930_999_CNRM
    Oth_999_NorESM = qctl_NOI_1985_2014_999_NorESM - qctl_NOI_1901_1930_999_NorESM
    Oth_999_CESM2 = qctl_NOI_1985_2014_999_CESM2 - qctl_NOI_1901_1930_999_CESM2
    Oth_999_CESM2_gw = qctl_NOI_1985_2014_999_CESM2_gw - qctl_NOI_1901_1930_999_CESM2_gw
    Oth_999_IPSL = qctl_NOI_1985_2014_999_IPSL - qctl_NOI_1901_1930_999_IPSL
    Oth_999_E3SM = qctl_NOI_1985_2014_999_E3SM - qctl_NOI_1901_1930_999_E3SM
    
    Oth_999_consis_pos = np.zeros([163, 288])
    Oth_999_consis_neg = np.zeros([163, 288])
    Oth_999_consis_pos[Oth_999_CNRM>=0.2] = Oth_999_consis_pos[Oth_999_CNRM>=0.2]+1
    Oth_999_consis_neg[Oth_999_CNRM<=-0.2] = Oth_999_consis_neg[Oth_999_CNRM<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_NorESM>=0.2] = Oth_999_consis_pos[Oth_999_NorESM>=0.2]+1
    Oth_999_consis_neg[Oth_999_NorESM<=-0.2] = Oth_999_consis_neg[Oth_999_NorESM<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_CESM2>=0.2] = Oth_999_consis_pos[Oth_999_CESM2>=0.2]+1
    Oth_999_consis_neg[Oth_999_CESM2<=-0.2] = Oth_999_consis_neg[Oth_999_CESM2<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_CESM2_gw>=0.2] = Oth_999_consis_pos[Oth_999_CESM2_gw>=0.2]+1
    Oth_999_consis_neg[Oth_999_CESM2_gw<=-0.2] = Oth_999_consis_neg[Oth_999_CESM2_gw<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_IPSL>=0.2] = Oth_999_consis_pos[Oth_999_IPSL>=0.2]+1
    Oth_999_consis_neg[Oth_999_IPSL<=-0.2] = Oth_999_consis_neg[Oth_999_IPSL<=-0.2]+1
    
    Oth_999_consis_pos[Oth_999_E3SM>=0.2] = Oth_999_consis_pos[Oth_999_E3SM>=0.2]+1
    Oth_999_consis_neg[Oth_999_E3SM<=-0.2] = Oth_999_consis_neg[Oth_999_E3SM<=-0.2]+1
    
    Oth_999_consis = np.maximum(Oth_999_consis_pos, Oth_999_consis_neg)
    

    data_xr_6 = xr.DataArray(qctl_NOI_1985_2014_999-qctl_NOI_1901_1930_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(4, 3, 9, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_6, Oth_999_consis, 'ALL-IE', varname, '99.9%')
    ax.text(0.01, 0.92, 'i', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')

    
    
    Irr_999_CNRM = qctl_IRR_1985_2014_999_CNRM - qctl_NOI_1985_2014_999_CNRM
    Irr_999_NorESM = qctl_IRR_1985_2014_999_NorESM - qctl_NOI_1985_2014_999_NorESM
    Irr_999_CESM2 = qctl_IRR_1985_2014_999_CESM2 - qctl_NOI_1985_2014_999_CESM2
    Irr_999_CESM2_gw = qctl_IRR_1985_2014_999_CESM2_gw - qctl_NOI_1985_2014_999_CESM2_gw
    Irr_999_IPSL = qctl_IRR_1985_2014_999_IPSL - qctl_NOI_1985_2014_999_IPSL
    Irr_999_E3SM = qctl_IRR_1985_2014_999_E3SM - qctl_NOI_1985_2014_999_E3SM
    
    Irr_999_consis_pos = np.zeros([163, 288])
    Irr_999_consis_neg = np.zeros([163, 288])
    Irr_999_consis_pos[Irr_999_CNRM>=0.2] = Irr_999_consis_pos[Irr_999_CNRM>=0.2]+1
    Irr_999_consis_neg[Irr_999_CNRM<=-0.2] = Irr_999_consis_neg[Irr_999_CNRM<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_NorESM>=0.2] = Irr_999_consis_pos[Irr_999_NorESM>=0.2]+1
    Irr_999_consis_neg[Irr_999_NorESM<=-0.2] = Irr_999_consis_neg[Irr_999_NorESM<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_CESM2>=0.2] = Irr_999_consis_pos[Irr_999_CESM2>=0.2]+1
    Irr_999_consis_neg[Irr_999_CESM2<=-0.2] = Irr_999_consis_neg[Irr_999_CESM2<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_CESM2_gw>=0.2] = Irr_999_consis_pos[Irr_999_CESM2_gw>=0.2]+1
    Irr_999_consis_neg[Irr_999_CESM2_gw<=-0.2] = Irr_999_consis_neg[Irr_999_CESM2_gw<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_IPSL>=0.2] = Irr_999_consis_pos[Irr_999_IPSL>=0.2]+1
    Irr_999_consis_neg[Irr_999_IPSL<=-0.2] = Irr_999_consis_neg[Irr_999_IPSL<=-0.2]+1
    
    Irr_999_consis_pos[Irr_999_E3SM>=0.2] = Irr_999_consis_pos[Irr_999_E3SM>=0.2]+1
    Irr_999_consis_neg[Irr_999_E3SM<=-0.2] = Irr_999_consis_neg[Irr_999_E3SM<=-0.2]+1
    
    Irr_999_consis = np.maximum(Irr_999_consis_pos, Irr_999_consis_neg)
    

    data_xr_9 = xr.DataArray(qctl_IRR_1985_2014_999-qctl_NOI_1985_2014_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(4,3,12, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_9, Irr_999_consis, 'IE', varname, '99.9%', unit)
    ax.text(0.01, 0.92, 'l', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')
    
    
    
    All_999_CNRM = qctl_IRR_1985_2014_999_CNRM - qctl_IRR_1901_1930_999_CNRM
    All_999_NorESM = qctl_IRR_1985_2014_999_NorESM - qctl_IRR_1901_1930_999_NorESM
    All_999_CESM2 = qctl_IRR_1985_2014_999_CESM2 - qctl_IRR_1901_1930_999_CESM2
    All_999_CESM2_gw = qctl_IRR_1985_2014_999_CESM2_gw - qctl_IRR_1901_1930_999_CESM2_gw
    All_999_IPSL = qctl_IRR_1985_2014_999_IPSL - qctl_IRR_1901_1930_999_IPSL
    All_999_E3SM = qctl_IRR_1985_2014_999_E3SM - qctl_IRR_1901_1930_999_E3SM
    
    All_999_consis_pos = np.zeros([163, 288])
    All_999_consis_neg = np.zeros([163, 288])
    All_999_consis_pos[All_999_CNRM>=0.2] = All_999_consis_pos[All_999_CNRM>=0.2]+1
    All_999_consis_neg[All_999_CNRM<=-0.2] = All_999_consis_neg[All_999_CNRM<=-0.2]+1
    
    All_999_consis_pos[All_999_NorESM>=0.2] = All_999_consis_pos[All_999_NorESM>=0.2]+1
    All_999_consis_neg[All_999_NorESM<=-0.2] = All_999_consis_neg[All_999_NorESM<=-0.2]+1
    
    All_999_consis_pos[All_999_CESM2>=0.2] = All_999_consis_pos[All_999_CESM2>=0.2]+1
    All_999_consis_neg[All_999_CESM2<=-0.2] = All_999_consis_neg[All_999_CESM2<=-0.2]+1
    
    All_999_consis_pos[All_999_CESM2_gw>=0.2] = All_999_consis_pos[All_999_CESM2_gw>=0.2]+1
    All_999_consis_neg[All_999_CESM2_gw<=-0.2] = All_999_consis_neg[All_999_CESM2_gw<=-0.2]+1
    
    All_999_consis_pos[All_999_IPSL>=0.2] = All_999_consis_pos[All_999_IPSL>=0.2]+1
    All_999_consis_neg[All_999_IPSL<=-0.2] = All_999_consis_neg[All_999_IPSL<=-0.2]+1
    
    All_999_consis_pos[All_999_E3SM>=0.2] = All_999_consis_pos[All_999_E3SM>=0.2]+1
    All_999_consis_neg[All_999_E3SM<=-0.2] = All_999_consis_neg[All_999_E3SM<=-0.2]+1
    
    All_999_consis = np.maximum(All_999_consis_pos, All_999_consis_neg)
    

    data_xr_12 = xr.DataArray(qctl_IRR_1985_2014_999-qctl_NOI_1901_1930_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(4,3,6, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_12, All_999_consis, 'ALL', varname, '99.9%')
    ax.text(0.01, 0.92, 'f', color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')

In [ ]:
all_for_paper()